# The Walking DEAD - um modelo de difusão social para o apocalipse zumbi!

Por uma série de motivos (inclua-se aqui meu problema de pesquisa de tese, curiosidade sobre ABM's, o vício da minha cunhada em Netflix) cheguei até o excelente artigo de Riebling e Schmitz (2016) intitulado **ZombieApocalypse: Modeling the social dynamics of infection and rejection.** O artigo é aberto e pode ser visualizado [aqui](http://journals.sagepub.com/doi/full/10.1177/2059799115622767). Sua ideia é bastante engenhosa: eles usam um modelo de difusão em redes (*network diffusion model*) para modelizar o alastrar de uma infecção zumbi generalizada. Neste post, vamos replicar uma versão simples do modelo tentando entender a implementação de um modelo de simulação computacional de agentes em redes. 

Os modelos de difusão são amplamente utilizados para estudar padrões de infecção e contágio na área da saúde utilizando técnicas da Análise de Redes Sociais (ARS). São um tipo de *Agent Based Modelin* ou Modelagem Baseada em Agentes, modelos onde agentes e suas ações são simulados computacionalmente. O [Rogério Barbosa](https://www.facebook.com/rogerio.barbosa.7528) tem uma série de posts muito bons sobre o método que podem ser vistos [aqui](https://sociaisemetodos.wordpress.com/2016/04/20/cachorros-artificiais-agentes-de-agent-based-modeling-usando-r-parte-1/). Na implementação desse tipo de modelos, o pesquisador programa as características dos agentes e do mundo social mais fundamentais de que precisa para tentar entender como os fenômenos no nível macro emergem das interações.

Para simular o *The Walking Dead*, vamos usar a distribuição Anaconda do Python 3.5. A distribuição pode ser baixada [aqui](https://www.continuum.io/downloads). Se é a primeira vez que você instala o Python, é importante instalar também alguns pacotes que não vem como *default* na distro Anaconda.

Se você é usuário de Windows, abra o prompt de comando clicando no botão do Windows e pesquisando por `cmd`. Se você é usuário de Ubuntu, abra o terminal com `Ctrl + Alt + t`. Para instalar os pacotes, use os comandos:

Vamos aos códigos.

## Carregando as bibliotecas

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from nxsim import NetworkSimulation, BaseNetworkAgent, BaseLoggingAgent

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set_context('notebook') #colocando um aspecto mais atrativo nos plots com seaborn

## Algumas funções úteis

In [2]:
def census_to_df(log, num_trials=1, state_id=1, dir_path='sim_01'):
    """Reads nxsim log files and returns the sum of agents with a given state_id at 
    every time interval of the simulation for every run of the simulation as a pandas
    DataFrame object."""
    D = {}
    for i in range(num_trials):
        name = 'Trial ' + str(i)
        trial = log.open_trial_state_history(dir_path=dir_path, 
                                             trial_id=i)
        census = [sum([1 for node_id, state in g.items() 
                       if node_id != 'topology' and state['id'] == state_id]) 
                  for t, g in trial.items()]
        D[name] = census      
    return pd.DataFrame(D)

def friends_to_the_end(log, num_trials=1, dir_path='sim_01'):
    """Reads nxsim log files and returns the number of human friends connected
    to every human at every time interval of the simulation for every run of
    the simulation as a pandas DataFrame object."""
    D = {}
    for i in range(num_trials):
        name = 'Trial ' + str(i)
        trial = log.open_trial_state_history(dir_path=dir_path, 
                                             trial_id=i)
        friends = [np.mean([state['friends'] for node_id, state in g.items() 
                       if node_id != 'topology' and state['id'] == 0]) 
                  for t, g in trial.items()]
        D[name] = friends    
    return pd.DataFrame(D)

def edge_count_to_df(log, num_trials=1, state_id=1, dir_path='sim_01'):
    """Reads nxsim log files and returns the edge count for the topology at every
    time interval of the simulation for every run of the simulation as a pandas
    DataFrame object."""
    D = {}
    for i in range(num_trials):
        name = 'Trial ' + str(i)
        trial = log.open_trial_state_history(dir_path=dir_path, 
                                             trial_id=i)
        edge_count = [len(trial[key]['topology']) for key in trial]
        D[name] = edge_count
    return pd.DataFrame(D)

## Topologia

Vamos declarar a topologia da rede sobre a qual as simulações irão acontecer. A principal diferença entre os modelos de difusão social e os ABM's comuns reside no pressuposto da independência das observações. Esse pressuposto não se sustenta em estruturas relacionais já que a mudança de apenas um laço muda toda a configuração da rede. O pacote *nxsim* nos permite rodar um modelo de simulações apenas dentro da topologia de uma rede dada. Vamos simular uma rede livre de escala com 100 nós, ou 100 pessoas.

In [3]:
number_of_nodes = 100
G = nx.scale_free_graph(number_of_nodes).to_undirected()

## Classe de agentes

Vamos agora definir a classe de agentes que nos interessa e as ações dos agentes. Nesse modelo, 